# Test: N qubti clifford Classical Shadow with Identity (Z-Basis)

In [ ]:
# Standard Imports
import sys
import numpy as np

# Qiskit Imports
from qiskit import QuantumCircuit, qasm2
from qiskit.quantum_info import Clifford, StabilizerState
from qiskit.visualization import array_to_latex
from qiskit_aer import AerSimulator

# Local Imports
sys.path.insert(0, "../../..")
from classical_shadow_n_clifford import ClassicalShadow_N_CLIFFORD
from shadow_protocol import ShadowProtocol

In [ ]:
# 1. Define Custom Shadow Class (Identity Only)
class IdentityShadow(ClassicalShadow_N_CLIFFORD):
    """
    Overrides random rotations to always return Identity (I).
    """
    def get_random_rotations(self, num_qubits) -> list[Clifford]:
        # Create an Identity Clifford (Circuit with no gates)
        qc_i = QuantumCircuit(num_qubits)
        c_i = Clifford(qc_i)
        return [c_i]


# 2. Define the Protocol
class ZeroStateProtocol(ShadowProtocol):

    def get_num_qubits(self) -> int:
        return 2
    
    def get_state_circuit(self) -> QuantumCircuit:
        # Returns a clean |00> state
        return QuantumCircuit(2)

    def run_circuit_and_get_measurement(self, circuit) -> list[int]:
        sim = AerSimulator()
        job = sim.run(circuit, shots=997)
        result = job.result()

        counts = result.get_counts()
        max_hits = max(counts, key=counts.get)
        bit_list = [int(bit) for bit in list(max_hits)]
        
        return bit_list[::-1] 
    
# 3. Instantiate
protocol = ZeroStateProtocol()
shadow_id = IdentityShadow(protocol)

In [ ]:
# Sanity Check: Ensure we only get Identity operators
found_clifford = {}

for i in range(0,1000):
    cliff = shadow_id.get_random_rotations(2)
    assert len(cliff) == 1
    circuit = cliff[0].to_circuit()
    program_as_string = qasm2.dumps(circuit)
    found_clifford[program_as_string] = cliff[0]

cliffs = list(found_clifford.values())
assert len(cliffs) == 1, "More than one type of Clifford was generated!"
print("Success: Only one Clifford operator (Identity) generated.")
display(cliffs[0].to_circuit().draw("mpl"))

### Visualizing the State and Circuit
Since we apply Identity, the circuit should just be the state preparation (empty circuit for $|00\rangle$) followed by measurement.

In [ ]:
state_circuit = shadow_id.shadow_protocol.get_state_circuit()
cliffords = shadow_id.get_random_rotations(2)

combined_circuit = shadow_id.make_rotated_state_circuit(
            cliffords, state_circuit
        )
display(combined_circuit.draw("mpl"))

### Verify Measurement Outcomes
We expect to strictly observe the outcome `00` (or `[0, 0]` as a list).

In [ ]:
output_set = set()

for i in range(100):
    measurement = shadow_id.shadow_protocol.run_circuit_and_get_measurement(combined_circuit)
    output_set.add(str(measurement))

assert len(output_set) == 1
print(f"All measurements returned: {list(output_set)[0]}")

### Theoretical Calculation

We define the starting matrix $X$ (representing the state $|00\rangle$) and apply the formula with $n=2$:

$$
X = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{pmatrix}, \quad n=2
$$

The reconstruction formula is:

$$
\mathcal{M}_n^{-1}(X) = (2^n + 1)X - \mathbb{I}
$$

Substituting $n=2$, the scaling factor becomes $(2^2 + 1) = 5$:

$$
\hat{\rho} = 5 \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{pmatrix} - \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix}
$$

Calculating the subtraction:

$$
\hat{\rho} = \begin{pmatrix} 5 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{pmatrix} - \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix}
=
\begin{pmatrix}
4 & 0 & 0 & 0\\
0 & -1 & 0 & 0\\
0 & 0 & -1 & 0\\
0 & 0 & 0 & -1
\end{pmatrix}
$$

In [ ]:
# Add one snapshot (which will be the deterministic case calculated above)
shadow_id.add_snapshot()

# TYPO FIXED: get_density_matrix (was desity)
dm = shadow_id.get_density_matrix_from_stabilizers()

display(array_to_latex(dm, prefix="Reconstructed \; \rho = "))